# COVID-19-FineTune-BERT-ResearchPapers-Semantic-Search

## Install packages

https://mccormickml.com/2019/07/22/BERT-fine-tuning/

https://github.com/google-research/bert/blob/master/extract_features.py#L235

https://www.reddit.com/r/LanguageTechnology/comments/b3umhq/d_how_to_fine_tune_bert_embeddings_to_your_own/


In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 870kB 57.3MB/s 
     |████████████████████████████████| 1.0MB 57.1MB/s 
     |████████████████████████████████| 3.7MB 66.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=4678b694a16e9da94de7e364eaa6efc28031d594b29ba2bbf9a3c5dd63ee1774
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 450kB 10.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=404fe1b7a418b4bf9f401c1b5af547dfbc8e94c3b57dce239e979df372587338
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
Successfully built sentence-transformers
  Found existing installation: transformers 2.8.0
    Uninstalling transformers-2.8.0:
      Successfully uninstalled transformers-2.8.0


In [0]:
!git clone https://github.com/huggingface/transformers.git
#https://github.com/huggingface/transformers/issues/2693

Cloning into 'transformers'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 23732 (delta 0), reused 2 (delta 0), pack-reused 23726
Receiving objects: 100% (23732/23732), 14.17 MiB | 31.55 MiB/s, done.
Resolving deltas: 100% (16801/16801), done.


## Download & Prepocess Data
or directly download preporcessed data from [here](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search#data-links) 

In [0]:
#install the kaggle data to google colab
#https://github.com/Kaggle/kaggle-api#api-credentials
!pip install kaggle
import os
!cp "/content/kaggle.json" /root/.kaggle
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge
!unzip  CORD-19-research-challenge.zip -d /content/CORD-19-research-challenge

built using
https://www.kaggle.com/maksimeren/covid-19-literature-clustering

In [0]:
import glob
import json
import pandas as pd
from tqdm import tqdm
root_path = '/content/CORD-19-research-challenge/'
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

In [0]:
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


### Read Data (Helpers)

In [0]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            try:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            except:
                print("no abstract")
                self.abstract.append("")
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

dcd7a1235ea74e3ef71d051103bf8a64c3c8f457: 12 Background 13 After the outbreak of novel coronavirus (2019-nCoV) starting in late 2019, a number 14 of researchers have reported the predicted the virus transmission dynamics. However, 15 under th... A novel coronavirus (2019-nCoV) appeared in December 2019 in Wuhan, Hubei 33 Province in central China had triggered city closure on Jan. 23, 2020, and lockdown 34 of all major cities in the province ...


In [0]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [0]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

Processing index: 0 of 29315
Processing index: 2931 of 29315
Processing index: 5862 of 29315
Processing index: 8793 of 29315
Processing index: 11724 of 29315
Processing index: 14655 of 29315
Processing index: 17586 of 29315
Processing index: 20517 of 29315
Processing index: 23448 of 29315
Processing index: 26379 of 29315
Processing index: 29310 of 29315


,paper_id,abstract,body_text,authors,title,journal,abstract_summary
0,dcd7a1235ea74e3ef71d051103bf8a64c3c8f457,12 Background 13 After the outbreak of novel c...,A novel coronavirus (2019-nCoV) appeared in De...,Xinhai Li. Xumao Zhao...,The lockdown of Hubei Province causing<br>dif...,NaN,12 Background 13 After the outbreak of novel<...
1,86b6b0c1b2777541feb83116bcb7a5cb12a52310,,Firstly informed to World Health Organization ...,"Jung, Y. J.. Park, G.-S....",Comparative analysis of primer-probe sets for...,NaN,Not provided.
2,73d80c8f5780d70bd8d343188c56e898e91557b6,Middle East respiratory syndrome coronavirus (...,Coronaviruses (CoVs) comprise a family of enve...,"Straus, M. R.. Tang, T....",Ca2+ ions promote fusion of Middle East<br>Re...,NaN,Middle East respiratory syndrome coronavirus<...
3,70cc2e5152d3dc4d44494124ff556c9bbe9e6f41,"1 Background: A new virus broke out in Wuhan, ...","In December 2019, a new type of unexplained pn...",Yafei Wang. Ying Zhou...,Clinical Characteristics of Patients with<br>...,NaN,"1 Background: A new virus broke out in Wuhan,..."
4,3b22eecad8a582436c52284a4db2198a98a94e18,The host antiviral response involves the induc...,Respiratory syncytial virus (RSV) belongs to t...,"Robitaille, A. C.. Caron, E....","DUSP1 regulates apoptosis and cell migration,...",NaN,The host antiviral response involves the<br>i...


### Handle Possible Duplicates

In [0]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

count     27663
unique    20191
top            
freq       7444
Name: abstract, dtype: object

In [0]:
df_covid['body_text'].describe(include='all')

count                                                 27663
unique                                                27662
top       In a global world, knowledge of imported infec...
freq                                                      2
Name: body_text, dtype: object

### Take a Look at the Data

In [0]:
df_covid.head()

,paper_id,abstract,body_text,authors,title,journal,abstract_summary
0,dcd7a1235ea74e3ef71d051103bf8a64c3c8f457,12 Background 13 After the outbreak of novel c...,A novel coronavirus (2019-nCoV) appeared in De...,Xinhai Li. Xumao Zhao...,The lockdown of Hubei Province causing<br>dif...,NaN,12 Background 13 After the outbreak of novel<...
1,86b6b0c1b2777541feb83116bcb7a5cb12a52310,,Firstly informed to World Health Organization ...,"Jung, Y. J.. Park, G.-S....",Comparative analysis of primer-probe sets for...,NaN,Not provided.
2,73d80c8f5780d70bd8d343188c56e898e91557b6,Middle East respiratory syndrome coronavirus (...,Coronaviruses (CoVs) comprise a family of enve...,"Straus, M. R.. Tang, T....",Ca2+ ions promote fusion of Middle East<br>Re...,NaN,Middle East respiratory syndrome coronavirus<...
3,70cc2e5152d3dc4d44494124ff556c9bbe9e6f41,"1 Background: A new virus broke out in Wuhan, ...","In December 2019, a new type of unexplained pn...",Yafei Wang. Ying Zhou...,Clinical Characteristics of Patients with<br>...,NaN,"1 Background: A new virus broke out in Wuhan,..."
4,3b22eecad8a582436c52284a4db2198a98a94e18,The host antiviral response involves the induc...,Respiratory syncytial virus (RSV) belongs to t...,"Robitaille, A. C.. Caron, E....","DUSP1 regulates apoptosis and cell migration,...",NaN,The host antiviral response involves the<br>i...


In [0]:
df_covid.describe()

,paper_id,abstract,body_text,authors,title,journal,abstract_summary
count,27663,27663,27663,26917,27619,26769,27663
unique,27663,20191,27662,25568,27240,3323,20184
top,ce248b901191d45f3e56f1e6664a0239738aa148,,"In a global world, knowledge of imported infec...","Domingo, Esteban",Index,PLoS One,Not provided.
freq,1,7444,2,14,68,1511,7444


### Data Pre-Process

In [0]:
df_covid.dropna(inplace=True)
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26043 entries, 885 to 27677
Data columns (total 7 columns):
paper_id            26043 non-null object
abstract            26043 non-null object
body_text           26043 non-null object
authors             26043 non-null object
title               26043 non-null object
journal             26043 non-null object
abstract_summary    26043 non-null object
dtypes: object(7)
memory usage: 1.6+ MB


In [0]:
df_covid = df_covid.head(12500)

In [0]:
import re

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [0]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: lower_case(x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: lower_case(x))

In [0]:
df_covid.head(4)

,paper_id,abstract,body_text,authors,title,journal,abstract_summary
885,db5333b01a10f165ae516d30f9d1fbf96ab4b841,footandmouth disease virus fmdv represses host...,footandmouth disease fmd an acute highly conta...,"Gao, Yuan. Sun, Shi-Qi...",Biological function of Foot-and-mouth<br>dise...,Virol J,Foot-and-mouth disease virus (FMDV)<br>repres...
886,335b0a3f21f764adcbe20ff71e422d823c410098,background gray wolves canis lupus were reintr...,several highmortality disease outbreaks among ...,"Almberg, Emily S.. Mech, L. David...",A Serological Survey of Infectious Disease in...,PLoS One,Background: Gray wolves (Canis lupus) were<br...
887,bad0e9f737316570c33138d5cc95cc233cd937ab,in niger acute respiratory infections aris are...,acute respiratory infections aris are responsi...,"Lagare, Adamou. Ousmane, Sani...",Molecular detection of respiratory pathogens<...,Health Sci Rep,"In Niger, acute respiratory infections (ARIs)..."
888,007bf75961da42a7e0cc8e2855e5c208a5ec65c1,the hemagglutininesterases hes envelope glycop...,to initiate infection viruses must bind to an ...,"Langereis, Martijn A.. Zeng, Qinghong...",The Murine Coronavirus<br>Hemagglutinin-ester...,PLoS Pathog,"The hemagglutinin-esterases (HEs), envelope<b..."


In [0]:
df_covid.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid.csv")

In [0]:
df_covid_test = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

,paper_id,abstract,body_text,title,abstract_summary
0,db5333b01a10f165ae516d30f9d1fbf96ab4b841,footandmouth disease virus fmdv represses host...,footandmouth disease fmd an acute highly conta...,Biological function of Foot-and-mouth<br>dise...,Foot-and-mouth disease virus (FMDV)<br>repres...
1,335b0a3f21f764adcbe20ff71e422d823c410098,background gray wolves canis lupus were reintr...,several highmortality disease outbreaks among ...,A Serological Survey of Infectious Disease in...,Background: Gray wolves (Canis lupus) were<br...
2,bad0e9f737316570c33138d5cc95cc233cd937ab,in niger acute respiratory infections aris are...,acute respiratory infections aris are responsi...,Molecular detection of respiratory pathogens<...,"In Niger, acute respiratory infections (ARIs)..."
3,007bf75961da42a7e0cc8e2855e5c208a5ec65c1,the hemagglutininesterases hes envelope glycop...,to initiate infection viruses must bind to an ...,The Murine Coronavirus<br>Hemagglutinin-ester...,"The hemagglutinin-esterases (HEs), envelope<b..."
4,d6a325260dac29bfe718f1e57160583cb23b5908,emerging evidence suggests that dipeptidyl pep...,the global burden of diabetes is escalating at...,The role of renal dipeptidyl peptidase-4 in<b...,Emerging evidence suggests that dipeptidyl<br...


In [0]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])

In [0]:
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

100%|██████████| 12500/12500 [00:00<00:00, 23067.45it/s]


In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences.csv")
df_sentences.head()

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences_Full.csv")
df_sentences.head()

,paper_id,title,abstract,abstract_summary
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 

## Preparing Data for Embedding

In [0]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [0]:
df_sentences.head()

,paper_id
Unnamed: 0,
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 end the orf region is generally divided into four functional areas l p1 p2 and p3 due to the different functions of mature polypeptides orfencoded polyprotein is processed into four products l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro structural proteins form the biological p

In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences_Full.csv", index_col=0)
df.head()

,paper_id,title,abstract,abstract_summary
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 

### Prepare input txt file

In [0]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]
print(len(df_sentences_list))
df_sentences_list[:5]

100%|██████████| 403341/403341 [00:00<00:00, 2201062.93it/s]

403341


['footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1]  upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth',
 'the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3]  the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4]  which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evo

In [0]:
file_content = "\n".join(df_sentences_list)
with open("input_text.txt","w") as f:
  f.write(file_content)
#with open("input_text.txt","w") as f:
#  f.write(file_content[:-9000])
#with open("test_text.txt","w") as f:
#  f.write(file_content[-9000:])

In [0]:
print(len(file_content[:-9000]))
print(len(file_content[-9000:]))

353940725
9000


## BERT

In [0]:
#export TRAIN_FILE=/path/to/dataset/wiki.train.raw
#export TEST_FILE=/path/to/dataset/wiki.test.raw
#https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/
#https://huggingface.co/transformers/v2.1.1/examples.html#roberta-bert-and-masked-language-modeling
!python "/content/transformers/examples/run_language_modeling.py" \
    --output_dir="/content/drive/My Drive/BertSentenceSimilarity/BERTfine" \
    --model_type=bert \
    --model_name_or_path=google/bert_uncased_L-2_H-128_A-2 \
    --do_train \
    --block_size=512 \
    --train_data_file="/content/input_text.txt" \
    --mlm

Streaming output truncated to the last 5000 lines.
Iteration:  49% 17703/36277 [09:50<10:11, 30.37it/s]
Iteration:  49% 17707/36277 [09:51<10:16, 30.15it/s]
Iteration:  49% 17711/36277 [09:51<10:13, 30.26it/s]
Iteration:  49% 17715/36277 [09:51<10:17, 30.04it/s]
Iteration:  49% 17719/36277 [09:51<10:18, 30.00it/s]
Iteration:  49% 17723/36277 [09:51<10:16, 30.12it/s]
Iteration:  49% 17727/36277 [09:51<10:19, 29.95it/s]
Iteration:  49% 17731/36277 [09:51<10:13, 30.22it/s]
Iteration:  49% 17735/36277 [09:52<10:10, 30.36it/s]
Iteration:  49% 17739/36277 [09:52<10:14, 30.16it/s]
Iteration:  49% 17743/36277 [09:52<10:12, 30.25it/s]
Iteration:  49% 17747/36277 [09:52<10:08, 30.45it/s]
Iteration:  49% 17751/36277 [09:52<10:06, 30.56it/s]
Iteration:  49% 17755/36277 [09:52<10:06, 30.54it/s]
Iteration:  49% 17759/36277 [09:52<10:06, 30.56it/s]
Iteration:  49% 17763/36277 [09:52<10:05, 30.58it/s]
Iteration:  49% 17767/36277 [09:53<10:08, 30.40it/s]
Iteration:  49% 17771/36277 [09:53<10:09, 30.39i

In [0]:
# if you want more RAMs from google colab
# fill your RAM using this simple loop
a = []
while(1):
    a.append('1')

In [0]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, losses
import scipy.spatial
import pickle as pkl
word_embedding_model = models.BERT("/content/drive/My Drive/BertSentenceSimilarity/BERTfine")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = model.encode(corpus,show_progress_bar=True)
with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_finetuned_embeddings.pkl" , "wb") as f:
  pkl.dump(corpus_embeddings,f)

In [0]:
len(corpus_embeddings)

403341

In [0]:
# Query sentences:
queries = ['What has been published about medical care?',
           'Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest',
           'Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually',
           'Resources to support skilled nursing facilities and long term care facilities.',
           'Mobilization of surge medical staff to address shortages in overwhelmed communities .',
           'Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies .']
query_embeddings = model.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")

Batches: 100%|██████████| 1/1 [00:00<00:00, 116.54it/s]



Top 5 most similar sentences in corpus:


==========================Query==============================
=== What has been published about medical care? =====
Score:    (Score: 0.8779) 

Paragraph:    what is already known about this topic what are the new findings 

paper_id:   f084dcc7e442ab282deb97670e1843e347cf1fd5 

Title:    Ebola Holding Units at government hospitals in<br>Sierra Leone: evidence for a flexible and effective<br>model for safe isolation, early treatment<br>initiation, hospital safety and health system functioning 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8735) 

Paragraph:    to identify how one health has been used recently in the medical literature 

paper_id:   f703b510de361a759c9fe3419fa3aeb092c95512 

Title:    One Health and Zoonoses: The Evolution of One<br>Health and Incorporation of Zoonoses 

Abstract:   nan 

Abstract_Summary:   Not provided. 

--------------------------------